# Day 16

## Part I

第一部分按照问题描述来编写逻辑即可。下面定义了两个类，一个类是代表区间对象Range，例如$[1-3]$之类的，定义__contains__方法使得可以直接用in运算符，方便写出易读易理解的代码。第二个类是ValidRanges，代表所有可能的区间，可以通过add_range方法为对象添加新的区间，添加的时候会合并所有可能重叠的区段，以提升最后判断的性能，同样也实现了__contains__方法方便使用in运算符。两个类的__repr__方法不是必须的，仅为调试时显示对象内容方便而设计：

In [1]:
from typing import List

class Range(object):
    def __init__(self, low: int, high: int):
        self.low = low
        self.high = high
    def set_low(self, low: int):
        self.low = low
    def set_high(self, high: int):
        self.high = high
    def __contains__(self, value: int) -> bool:
        return self.low <= value <= self.high
    def __repr__(self) -> str:
        return f'[{self.low} - {self.high}]'
    def __eq__(self, other: 'Range') -> bool:
        return self.low == other.low and self.high == other.high

class ValidRanges(object):
    def __init__(self):
        self.ranges: List[Range] = []
    def add_range(self, low: int, high: int):
        if low >= high:
            raise ValueError('invalid params: {low} >= {high}')
        for i, r in enumerate(self.ranges):
            if low in r and high in r:
                return
            if low in r or r.high == low - 1:
                r.set_high(high)
                return
            if high in r or r.low == high + 1:
                r.set_low(low)
                return
            if r.low > low and r.high < high:
                r.set_low(low)
                r.set_high(high)
                return
        self.ranges.append(Range(low, high))
    def __contains__(self, value: int) -> bool:
        return any(value in r for r in self.ranges)
    def __repr__(self) -> str:
        return str(self.ranges)

为这两个类做一个简单的单元测试：

In [2]:
v = ValidRanges()
v.add_range(1, 3)
v.add_range(5, 7)
v.add_range(6, 11)
v.add_range(33, 44)
v.add_range(13, 40)
v.add_range(45, 50)
assert(v.ranges == [Range(1, 3), Range(5, 11), Range(13, 50)])

读取输入数据，这是一项体力活，就是一项一项去读取解析：

In [3]:
from typing import Tuple

def read_input(input_file: str) -> Tuple[ValidRanges, List[int], List[List[int]]]:
    with open(input_file) as fn:
        data = fn.read()
    # 两个换行符分隔，第一部分是字段定义，第二部分是自己的票，第三部分是隔壁的票
    fields, your_ticket, nearby_tickets = data.split('\n\n')
    # 下面这个for loop将第一部分输入转换成一个ValidRanges对象
    valid_ranges = ValidRanges()
    for line in fields.split('\n'):
        _, f = line.rstrip().split(': ')
        for s in f.split(' or '):
            low, high = s.split('-')
            valid_ranges.add_range(int(low), int(high))
    # 下面使用两个列表解析式获得自己的票（一个整数列表）和隔壁的票（一个整数列表的列表）
    your_ticket = [int(x) for x in your_ticket.split('\n')[1].rstrip().split(',')]
    nearby_tickets = [[int(x) for x in line.rstrip().split(',')]
                      for line in nearby_tickets.split('\n')[1:] if line]
    return valid_ranges, your_ticket, nearby_tickets

ok，第一部分结果获得的逻辑：

In [4]:
def part1_solution(valid_ranges: ValidRanges, nearby_tickets: List[List[int]]) -> int:
    rate = 0
    for ticket in nearby_tickets:
        rate += sum(x for x in ticket if x not in valid_ranges)
    return rate

单元测试：

In [5]:
test_ranges, _, test_nearby = read_input('testcase1.txt')
assert(part1_solution(test_ranges, test_nearby) == 71)

然后就是第一部分的结果：

In [6]:
valid_ranges, _, nearby_tickets = read_input('input.txt')
part1_solution(valid_ranges, nearby_tickets)

24980

## Part II

看到第二部分的题目后，才发现第一部分的ValidRanges不应该那么设计，不过没关系，通过继承，可以实现一个新的ValidField类，只是为ValidRanges增加了一个属性field_name，代表这个字段的名称，也就是每一行的名称部分内容：

In [7]:
class ValidField(ValidRanges):
    def __init__(self, field_name: str):
        super().__init__()
        self.field_name = field_name
    def __repr__(self) -> str:
        return f'{self.field_name}: {super().__repr__()}'

当然读取输入数据的方式也要发生改变，下面就重新定义read_input函数，在返回的元组中多增加一个值，名称和ValidField组成的一个字典：

In [8]:
from typing import Dict

def read_input(input_file: str) \
    -> Tuple[ValidRanges, Dict[str, ValidField], List[int], List[List[int]]]:
    with open(input_file) as fn:
        data = fn.read()
    fields, your_ticket, nearby_tickets = data.split('\n\n')
    valid_ranges = ValidRanges()
    valid_fields = {}
    for line in fields.split('\n'):
        field_name, f = line.rstrip().split(': ')
        valid_field = ValidField(field_name)
        for s in f.split(' or '):
            low, high = s.split('-')
            valid_field.add_range(int(low), int(high))
            valid_ranges.add_range(int(low), int(high))
        valid_fields[field_name] = valid_field
    your_ticket = [int(x) for x in your_ticket.split('\n')[1].rstrip().split(',')]
    nearby_tickets = [[int(x) for x in line.rstrip().split(',')]
                      for line in nearby_tickets.split('\n')[1:] if line]
    return valid_ranges, valid_fields, your_ticket, nearby_tickets

然后我们需要移除隔壁票中所有不满足第一部分要求的那些票，定义一个函数来处理：

In [9]:
def remove_invalid_nearby_tickets(valid_ranges: ValidRanges,
                                  nearby_tickets: List[List[int]]) -> List[List[int]]:
    return [ticket for ticket in nearby_tickets if all(x in valid_ranges for x in ticket)]

使用第一部分的例子做一个单元测试，看看read_input和remove_invalid_nearby_tickets函数是否正常工作：

In [11]:
test_ranges, _, _, test_nearby = read_input('testcase1.txt')
assert(remove_invalid_nearby_tickets(test_ranges, test_nearby) == [[7, 3, 47]])

下面才是第二部分的关键逻辑，我们要找到票中的每一列对应的字段。首先找到票中每一列可能从属的字段列表，如果列表中只有一个可能性，则表示这个列只能对应该字段，并在其他列中将这个字段（如果存在的话）从列表中删除，重复这个过程直到所有的列都只对应唯一的一个字段为止，最终返回一个字段名称和列序号的字典值：

In [12]:
from typing import Set
import numpy as np

def remove_all_indices(indices: Dict[str, List[int]], value: int):
    '''
    在所有字段对应列序号的列表中删除一个特定序号值
    '''
    for v in indices.values():
        # 如果该字段就是只对应这个序号，或者该字段对应的序号列表中不存在value，跳过
        if v == [value] or value not in v:
            continue
        v.remove(value)

def find_field_indices(valid_fields: Dict[str, ValidField],
                       nearby_tickets: List[List[int]]) -> Dict[str, int]:
    # 这里借用了numpy的矩阵转置能力，直接操作列表的列表中的列也是可以的，不过代码会比较长且难读
    tickets = np.array(nearby_tickets).T
    indices = {}
    # 首先将所有字段可能出现的列序号找到，放到一个字典中
    for name in valid_fields:
        for i, col in enumerate(tickets):
            if all(x in valid_fields[name] for x in col):
                indices.setdefault(name, [])
                indices[name].append(i)
    result = {}
    # 然后通过不停的循环，将单个序号的字段放入result中，并调用remove_all_indices函数在其他字段中删除这个序号
    while True:
        # 直到所有字段都仅对应一个序号，循环结束
        if len(result) == len(indices):
            return result
        for name, inds in indices.items():
            if len(inds) == 1:
                result[name] = inds[0]
                remove_all_indices(indices, inds[0])

使用第二部分给出的例子，做上面逻辑的单元测试：

In [13]:
test_ranges, test_fields, _, test_nearby = read_input('testcase2.txt')
test_nearby = remove_invalid_nearby_tickets(test_ranges, test_nearby)
assert(find_field_indices(test_fields, test_nearby) == {'row': 0, 'class': 1, 'seat': 2})

最后的函数，用来获得第二部分的结果，自己的票中6个以departure开头的字段值的乘积：

In [14]:
def part2_solution(valid_ranges: ValidRanges,
                   valid_fields: Dict[str, ValidField],
                   your_ticket: List[int],
                   nearby_tickets: List[List[int]]) -> int:
    # 移除不满足Part I条件的那些票
    nearby_tickets = remove_invalid_nearby_tickets(valid_ranges, nearby_tickets)
    # 找到所有字段对应的列序号
    indices = find_field_indices(valid_fields, nearby_tickets)
    # 下面计算乘积，此处没有使用reduce，用for loop，懒得import reduce了
    result = 1
    for index in (index for name, index in indices.items() if name.startswith('departure')):
        result *= your_ticket[index]
    return result

第二部分结果：

In [15]:
valid_ranges, valid_fields, your_ticket, nearby_tickets = read_input('input.txt')
part2_solution(valid_ranges, valid_fields, your_ticket, nearby_tickets)

809376774329